# 8. 미국 주식 데이터 수집하기

퀀트 투자의 장점은 데이터만 있다면 동일한 투자 전략을 전세계 모든 국가에 적용할 수 있다는 점입니다. 이번 장에서는 미국 전 종목의 티커 수집 및 주가, 재무제표, 가치지표를 다운로드 해보도록 하겠습니다.

## 8.1 티커 수집하기

우리나라는 거래소가 '한국거래소' 한 곳 뿐이지만, 미국은 매우 많은 거래소가 존재합니다. 따라서 동일한 종목이 여러 거래소에서 거래되는 일도 발생합니다. 그 중에서도 가장 규모가 큰 거래소는 뉴욕거래소(NYSE), 나스닥(NASDAQ), 아멕스(AMEX)이며, 해당 거래소에 상장된 종목들은 나스닥 홈페이지에서 제공됩니다.

```
https://www.nasdaq.com/market-activity/stocks/screener
```

위 사이트에 접속한 후, 하단의 [Download CSV] 버튼을 누르면, 해당 데이터를 받아오는 웹주소가 표시됩니다.

```{figure} image/us_ticker.png
---
name: us_ticker
---
미국의 상장종목 리스트
```

```
https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=25&offset=0&download=true
```

해당 주소에 접속하면, 각 종목에 대한 정보가 JSON 형태로 제공됩니다. 이를 크롤링 해보도록 하겠습니다.

In [1]:
import pandas as pd
import requests as rq

url = 'https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=25&offset=0&download=true'
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

data = rq.get(url, headers=headers)
data_json = data.json()['data']
all_data = pd.DataFrame(data = data_json['rows'], columns = data_json['headers'])

all_data.head()

,symbol,name,lastsale,netchange,pctchange,marketCap,country,ipoyear,volume,sector,industry,url
0,A,Agilent Technologies Inc. Common Stock,$131.91,1.22,0.934%,40192649204.00,United States,1999,1552194,Capital Goods,Biotechnology: Laboratory Analytical Instruments,/market-activity/stocks/a
1,AA,Alcoa Corporation Common Stock,$31.36,-0.09,-0.286%,5843032424.00,,2016,4315627,Basic Industries,Aluminum,/market-activity/stocks/aa
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.8752,-0.0148,-0.15%,1234523440.00,,2021,18355,Finance,Business Services,/market-activity/stocks/aac
3,AACG,ATA Creativity Global American Depositary Shares,$4.06,0.00,0.00%,127283034.00,China,,82032,Consumer Services,Other Consumer Services,/market-activity/stocks/aacg
4,AACQ,Artius Acquisition Inc. Class A Common Stock,$10.30,-0.09,-0.866%,932793750.00,United States,2020,1022952,Basic Industries,Major Chemicals,/market-activity/stocks/aacq


1. 먼저 url과 헤더정보를 입력하며, 헤더의 User-Agent에는 웹브라우저 구별을 입력해줍니다. 해당 사이트는 크롤러와 같이 정체가 불분명한 웹브라우저를 통한 접속이 막혀 있어, 마치 모질라 혹은 크롬을 통해 접속한 것 처럼 데이터를 요청합니다. 다양한 웹브라우저 리스트는 다음 사이트에서 확인할 수 있습니다. 
http://www.useragentstring.com/pages/useragentstring.php
2. `get()` 함수를 통해 페이지의 데이터를 불러옵니다.
3. `json()` 함수를 통해 json 형태로 불러온 후, data 부분을 선택합니다.
4. json 형태를 데이터프레임 형태로 변경해줍니다. 데이터는 json의 'rows'에 해당하는 부분을, 열 이름은 'headers'에 해당하는 부분을 입력합니다.

In [2]:
len(all_data)

7526

총 종목수를 살펴보면 상당히 많은 종목이 존재합니다. 그러나 동일한 종목이 여러 거래소에 상장된 경우가 있으므로, 이를 먼저 제거해주도록 합니다. symbol과 name열 중 중복되는 종목이 있으면 첫번쨰 데이터만을 남기도록 하겠습니다.

In [3]:
all_data = all_data.loc[~all_data.symbol.duplicated(keep = 'first'), :]
all_data = all_data.loc[~all_data.name.duplicated(keep = 'first'), :]

### 8.1.1 티커 클렌징

위에서 받은 데이터 중 모든 종목이 투자에 적합한 종목이 아니므로 클렌징 처리를 해주도록 합니다. 그 기준과 순서는 다음과 같습니다.

1. marketCap이 0인 종목 제거 (스팩 주식)
2. 종목명에 '^' 들어간 종목 제거 (우선주)
3. Preferred가 들어간 종목 제거 (우선주)
4. ETF 혹은 펀드 제거
5. 종목명에 '%'이 들어간 종목 제거 (우선주)
6. 종목명에 '/'를 '.' 으로 변경
7. 티커 순으로 정렬

In [4]:
all_data['marketCap'] = pd.to_numeric(all_data['marketCap'])
all_data = all_data.loc[all_data['marketCap'] != 0]
all_data = all_data.loc[~all_data['symbol'].str.contains('\\^')]
all_data = all_data.loc[~all_data['name'].str.contains('Preferred')]
all_data = all_data.loc[~all_data['name'].str.contains('preferred')]
all_data = all_data.loc[~all_data['name'].str.contains('ETF')]
all_data = all_data.loc[~all_data['name'].str.contains('Fund')]
all_data = all_data.loc[~all_data['name'].str.contains('%')]
all_data['symbol'] = all_data['symbol'].str.replace('\\/', '-')
all_data = all_data.sort_values(by = 'symbol')
all_data = all_data.reset_index(drop=True)

len(all_data)

5554

종목수가 현격하게 줄어들었습니다. 물론 해당 과정을 거쳐도 다중 클래스가 상장되어 있는 등 클렌징 처리를 할 종목이 많지만 그 수가 매우 적으므로, 효율성 측면에서 위의 과정 정도만으로도 충분합니다. 마지막으로 csv 파일로 저장해주도록 하겠습니다.

In [5]:
all_data.to_csv('data/US_ticker.csv')

## 8.2 주가 다운로드

미국 주가의 경우 3장에서 살펴본 `DataReader()` 함수를 통한 야후 API를 통해 매우 손쉽게 다운로드 받을 수 있습니다. 전반적인 과정은 한국의 전 종목 주가 크롤링과 매우 유사합니다.

In [ ]:
import pandas as pd
import numpy as np
import os
import pandas_datareader as web
from datetime import date
import time
from tqdm import tqdm

US_ticker = pd.read_csv('data/US_ticker.csv', index_col=0)
error_list = []

if not os.path.exists('data/US_price'):
    os.makedirs('data/US_price')
    
for i in tqdm(range(0, len(US_ticker))): 

    # 빈 데이터프레임 생성
    price = pd.DataFrame({'Price' : [np.nan]})
    price.index = [date.today().strftime("%Y-%m-%d")]
    
    # 티커 선택
    name = US_ticker['symbol'][i]  
    
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
                
        # 데이터 다운로드
        price = web.DataReader(name, 'yahoo')     
        
    
    except:
        # 오류 발생시 해당 종목명을 저장하고 다음 루프로 이동        
        error_list.append(name)     
    
    # 다운로드 받은 파일을 생성한 폴더 내 csv 파일로 저장
    price.to_csv('data/US_price/'+name+'_price.csv')
    
    # 타임슬립 적용
    time.sleep(2)    

## 8.3 재무제표 및 가치지표 크롤링

미국 주식의 재무제표를 구할 수 있는 사이트는 여러곳 있지만, 가장 유명한 곳은 야후 파이낸스 입니다.

```
https://finance.yahoo.com/
```

해당 사이트에는 종목에 대한 다양한 정보가 있기에 크롤링을 통해 데이터를 수집할 수 있습니다. 그러나 해당 페이지를 크롤링 하는 작업 역시 이미 여러 패키지들에 의해 개발되어 있으며, 책에서는 그 중에서 `yahoo_fin` 패키지를 사용하도록 하겠습니다. 해당 패키지의 자세한 설명은 아래 사이트에서 확인할 수 있습니다.

```
http://theautomatic.net/yahoo_fin-documentation/
```

### 8.3.1 재무제표 다운로드

먼저 재무제표를 다운로드 받도록 하겠습니다. `get_financials()` 함수를 통해 손익계산서, 재무상태표, 현금흐름표를 한번에 다운로드 받을 수 있으며, 예시로 애플(AAPL) 종목의 연간 재무제표를 받아보도록 하겠습니다.

In [6]:
import yahoo_fin.stock_info as si

data = si.get_financials('AAPL', yearly = True, quarterly = False)

data.keys()

dict_keys(['yearly_income_statement', 'yearly_balance_sheet', 'yearly_cash_flow'])

딕셔너리 형태로 세 종류의 재무제표가 다운로드 되었습니다. 이를 하나의 데이터프레임으로 바꿔주도록 하겠습니다.

In [7]:
data_fs = pd.concat([v for k, v in data.items()])

data_fs.head()

endDate,2020-09-26,2019-09-28,2018-09-29,2017-09-30
Breakdown,,,,
researchDevelopment,18752000000,16217000000,14236000000,11581000000
effectOfAccountingCharges,None,None,None,None
incomeBeforeTax,67091000000,65737000000,72903000000,64089000000
minorityInterest,None,None,None,None
netIncome,57411000000,55256000000,59531000000,48351000000


재무제표의 계정명과 순서는 홈페이지와 다소 다르지만, 값들을 비교해보면 동일한 데이터임을 확인할 수 있습니다. 이처럼 패키지를 이용하여 미국 데이터도 매우 쉽게 다운로드 받을 수 있습니다.

```{note}
분기별 재무제표 데이터를 받고자 할 경우 다음과 같이 입력하면 됩니다.

`si.get_financials('AAPL', yearly = False, quarterly = True)`
```

### 8.3.2 가치지표 다운로드

해당 패키지의 `get_stats_valuation()` 함수를 이용하면 홈페이지에 계산된 여러 가치지표 역시 손쉽게 다운로드 받을 수 있습니다. 

In [8]:
import yahoo_fin.stock_info as si

data_value = si.get_stats_valuation('AAPL')
data_value = data_value.set_index(data_value.columns[0])

data_value

,1
0,
Market Cap (intraday) 5,2.23T
Enterprise Value 3,2.27T
Trailing P/E,36.07
Forward P/E 1,28.30
PEG Ratio (5 yr expected) 1,1.84
Price/Sales (ttm),7.59
Price/Book (mrq),33.79
Enterprise Value/Revenue 3,7.71
Enterprise Value/EBITDA 7,26.63


PER, PEG, PSR, PBR 등 다양한 가치지표가 계산되어 있습니다. 이 중 지표명 끝의 숫자는 홈페이지에서 각주에 해당하는 불필요한 데이터이므로, 모든 숫자를 제거해주도록 하겠습니다.

In [9]:
data_value.index = data_value.index.str.replace('\d+', '')

data_value

,1
0,
Market Cap (intraday),2.23T
Enterprise Value,2.27T
Trailing P/E,36.07
Forward P/E,28.30
PEG Ratio ( yr expected),1.84
Price/Sales (ttm),7.59
Price/Book (mrq),33.79
Enterprise Value/Revenue,7.71
Enterprise Value/EBITDA,26.63


### 8.3.3 전 종목 재무제표 및 가치지표 다운로드

위 코드에서 for loop 구문을 이용해 티커에 해당하는 값만 변경해주면 모든 종목의 재무제표와 가치지표를 다운로드 받을 수 있습니다. 해당 코드는 다음과 같습니다.

In [ ]:
import pandas as pd
import numpy as np
import os
import yahoo_fin.stock_info as si
import time
from tqdm import tqdm

headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# 폴더 만들기
if not os.path.exists('data/US_fs'):
    os.makedirs('data/US_fs')
if not os.path.exists('data/US_value'):
    os.makedirs('data/US_value')

# 티커 불러오기
US_ticker = pd.read_csv('data/US_ticker.csv', index_col=0)
    
# for loop
for i in tqdm(range(0, len(US_ticker))):   
    
    # 빈 데이터프레임 생성
    data_fs = pd.DataFrame({'' : [np.nan]})    
    data_value = pd.DataFrame({'' : [np.nan]})    
    
    # 티커 선택
    name = US_ticker['symbol'][i]    
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
        # 재무제표 다운로드 
        data = si.get_financials(name, yearly = True, quarterly = False)
        data_fs = pd.concat([v for k, v in data.items()])
        
        # 가치지표 다운로드
        data_value = si.get_stats_valuation(name)
        data_value = data_value.set_index(data_value.columns[0])
        data_value.index = data_value.index.str.replace('\d+', '')          
        
    except:
        # 오류 발생시 해당 종목명을 저장하고 다음 루프로 이동        
        error_list.append(name)     
         
    
    # 다운로드 받은 파일을 생성한 폴더 내 csv 파일로 저장    
    data_fs.to_csv('data/US_fs/'+name+'_fs.csv')    
    data_value.to_csv('data/US_value/'+name+'_value.csv')    
    
    # 타임슬립 적용
    time.sleep(2)

```{note}
미국 종목의 재무제표 및 가치지표를 크롤링 할 때는 finviz 역시 많이 사용됩니다. 책에서 배운 크롤링 지식을 토대로 해당 페이지의 데이터도 한번 크롤링 해 보시길 바랍니다.

https://finviz.com/
    
```  